In [110]:
import pandas as pd
import numpy as np
from datetime import datetime
import datetime as dt

In [299]:
# read excel with country names and codes
# return list with country codes relevant for UCDP
country_code_pathfile = '/Users/sabinejoseph/Downloads/kfe-sabinejo-patch-1/Input/Country_codes_NAMO.xlsx'
country_code_column_name = 'UCDP_country_codes'
sheet = 'Sheet1'

def country_codes_from_excel(country_codes, sheet_num, column_name):
    countries = pd.ExcelFile(country_codes)
    countries = countries.parse(sheet_num)
    return list(countries[column_name]) # UCDP uses Gleditsch and Ward country codes

CCS = country_codes_from_excel(country_code_pathfile, sheet, country_code_column_name)
CC3 = country_codes_from_excel(country_code_pathfile, sheet, 'Country_3')

In [300]:
UCDP_filename = '/Users/sabinejoseph/Downloads/kfe-sabinejo-patch-1/ged50-csv/ged50.csv'
country_code_column_name = 'gwno'

def csv_to_df(filename, country_codes, csv_country_code_column):
    df = pd.read_csv(filename , sep=',')
    return df[df[csv_country_code_column].isin(country_codes)]

df = csv_to_df(UCDP_filename, CCS, country_code_column_name)

In [301]:
# delete not to be used columns
# 'id' # only keep for data cleaning 
vars_to_del = ['relid', 'year', 'active_year', 'conflict_dset_id', 'conflict_name', 'dyad_dset_id',
               'dyad_new_id', 'dyad_name', 'side_a_dset_id', 'side_a_new_id', 'side_a', 'side_b_dset_id',
               'side_b_new_id', 'side_b', 'source_date', 'source_headline', 'source_office', 'source_original',
               'where_prec', 'where_coordinates', 'where_description', 'adm_1', 'adm_2', 'latitude', 'longitude',
               'geom_wkt', 'priogrid_gid', 'country', 'region', 'isocc', 'gwab']

def del_columns_from_df(col_names):
    for i in col_names:
        del df[i]
    return df

df = del_columns_from_df(vars_to_del)

In [302]:
# str index to datetime
df = df.reset_index(drop=True)

df_datestring_column_name = 'date_start'
dateformat = '%Y-%m-%d'

def str_to_datetime(col_name, dateformat):
    return [datetime.strptime(str(df[col_name][i]), dateformat) 
            for i in range(0, len(df[col_name])) if i is not None]

df[df_datestring_column_name] = str_to_datetime(df_datestring_column_name, dateformat)


In [ ]:
# filter by type of crisis --- check with Oliver
# type_of_violence # 1: state-based conflict # 2: non-state conflict # 3: one-sided violence
# df = df[df['type_of_violence'].isin([1, 3])]


# df = df[df.date_prec > 1] 

# ? conflict_new_id #as filter? -- existing conflict -- ask Oliver
#df.loc[(df['conflict_new_id'] == 426) & df['type_of_violence'].isin([1, 3]))]
#df.loc[(df[' '].isin([426, 234]) & df['type_of_violence'].isin([1, 3]))]

# filter by event_clarity?

### to use:
# date var: 
# combination of: date_start, date_end
# date_prec #duration of event, only chose date_prec = 1 (verify with Oliver?)
# df = df.loc[(df['date_prec'] == 1)]

# -> date_start = date_end - chose as event date
# or always chose date_start as date
# del df.date_end

# number_of_sources #importance - only valid for yr 2013 and 2014
# del df.event_clarity


In [304]:
# grouping by date per country code
new_format = '%Y-%m-%d'
col_name_date = 'date_start'
col_name_country_codes = 'gwno'
agg_col_names = ['deaths_a', 'deaths_b', 'deaths_civilians', 'deaths_unknown', 'best_est', 'high_est', 'low_est']
event_count_col_name = 'count_num_daily_events'

def group_by_country_code_date_agg_sum(date, CC, col_name_list, ct_col_name):
# sum of death counts
# count of events per day per country code
    df[ct_col_name] = 1 
    col_name_list.append(ct_col_name)
    return df.groupby([date, CC]).agg(dict.fromkeys(col_name_list, sum))
    
df_agg = group_by_country_code_date_agg_sum(col_name_date, col_name_country_codes, agg_col_names, event_count_col_name).reset_index()
df_agg.date_start = [df_agg.date_start[i].strftime(new_format) for i in range(0, len(df_agg.index)) if i is not None]
df_agg[:5]

#df_agg.to_csv('Agg_UCDP_daily.csv')

,date_start,gwno,count_num_daily_events,deaths_civilians,deaths_unknown,best_est,high_est,low_est,deaths_a,deaths_b
0,1989-01-01,640,1,0,0,16,16,16,16,0
1,1989-01-01,645,2,0,50,50,50,50,0,0
2,1989-01-01,660,8,5,32,40,217,40,2,1
3,1989-01-02,660,2,0,9,9,9,9,0,0
4,1989-01-08,660,6,0,27,28,28,28,1,0


In [280]:
#### new notebook from here 
# create empty df
#old_format = '%Y-%m-%d %H:%M:%S'
start_date = df_agg['date_start'].min() #'2014-01-01 00:00:00'
end_date = df_agg['date_start'].max() #'2017-08-01 00:00:00'
new_CC_col_name = 'CC3'
new_CC_col_name2 = 'CCS'
#df_name = df

def create_empty_df(date_format, start_date, end_date, CC_col_name, new_col_name2):
    all_days = pd.date_range(start_date, end_date, freq='D')

    df = pd.DataFrame(index=pd.date_range(start = start_date, end = end_date, freq='90Min'))
    df.index = [df.index[i].strftime(new_format) for i in range(0, len(df.index)) if i is not None]

    #df[CC_col_name] = np.nan
    new_CC3 = CC3 * (len(df.index)/len(CC3))
    new_CCS = CCS * (len(df.index)/len(CCS))
    df = df[:-1]

    df[CC_col_name] = new_CC3
    df[new_col_name2] = new_CCS
    df.is_copy = False
    return df

df = create_empty_df(new_format, start_date, end_date, new_CC_col_name, new_CC_col_name2)
df = df.reset_index()
df = df.rename(columns={'index': 'date_start'})

In [282]:
cols_to_append = ['count_num_daily_events', 'deaths_civilians', 'deaths_unknown', 'best_est', 'high_est',
                  'low_est', 'deaths_a', 'deaths_b']

def append_empty_cols_to_df(df, col_names):
    for i in cols_names:
        df[i] = np.nan
        return df
        
df = append_empty_cols_to_df(df, cols_to_append)

In [291]:
# very slow, still performs if we only select relevant time frame?

def combine_empty_and_filled_df(df, df_agg, CC3, CCS, cols_to_append):
    for i in range(0, len(df)):
    #for i in range(0, 100):
        if df.CC3[i] != 'PSE':
            df_temp = df_agg[(df_agg.date_start == df.date_start[i]) & (df_agg.gwno == int(df.CCS[i]))]
            #to list 
            if df_temp.empty is False:
                #list to row
                for k in cols_to_append:
                    df[k][i] = df_temp[k]
                    df.is_copy = False
                    #df.loc[i, k] = df_temp[k] #--> bug
    return df
                    

combined_df = combine_empty_and_filled_df(df, df_agg, CC3, CCS, cols_to_append)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [297]:
# subset df based on date
df = df[(df['date_start'] > '2014-01-01') & (df['date_start'] <= '2017-08-01')]
df.shape

(11648, 11)

In [209]:
#df = df.rename(columns={'date_start': 'date'})

#merged = df.merge(df_agg, how='left', left_on='date', right_on='date_start')
#merged[:50]

#df.join(df_agg, lsuffix='_df', rsuffix='_df_agg')

In [210]:
#df.set_index('date').join(df_agg.set_index('date_start'))

In [ ]:
#df1=df
#df_agg1=df_agg

#df1.set_index(df1['date'], inplace=True)
#df_agg1.set_index(df_agg1['date_start'], inplace=True) 

#df1.join(df_agg1, lsuffix='_df1', rsuffix='_df_agg1')

#test = pd.concat([df, df_agg], axis=1, ) #both dfs must have same length 
#test[:30]
#merge=pd.merge(df, df_agg, how='left', left_index=True, right_index=True)
#merge[:30]

#merged = df.merge(df_agg, left_on='date_start', right_on='date_start', how='outer')